In [1]:
import os
import pandas as pd
from tqdm import tqdm

# 원본 CSV 파일들이 있는 소스 디렉토리
source_dir = '법령별 csv 저장 중복제거'

# 생성된 .txt 파일들을 저장할 출력 디렉토리
output_dir = '법령_통합텍스트(현역)'

# 합칠 대상 컬럼 목록
# ★ 만약 다른 컬럼도 추가하고 싶다면 이 리스트에 추가하세요.
columns_to_combine = ['조문내용', '항', '호', '목']

# 출력 디렉토리 생성 (이미 존재하면 넘어감)
os.makedirs(output_dir, exist_ok=True)

print(f"'{source_dir}' 폴더의 CSV 파일들을 읽어 '{output_dir}' 폴더에 TXT 파일로 통합합니다.")
print(f"통합 대상 컬럼: {columns_to_combine}")

# 소스 디렉토리의 모든 파일에 대해 반복 작업
if not os.path.isdir(source_dir):
    print(f"오류: 원본 폴더 '{source_dir}'를 찾을 수 없습니다. 경로를 확인해주세요.")
else:
    for filename in tqdm(os.listdir(source_dir)):
        if filename.endswith('.csv'):
            try:
                source_path = os.path.join(source_dir, filename)
                df = pd.read_csv(source_path, sep='\t')

                # 실제 파일에 존재하는 컬럼만 필터링
                existing_columns = [col for col in columns_to_combine if col in df.columns]

                # 해당 컬럼들의 빈 값(NaN)을 빈 문자열('')로 대체
                df[existing_columns] = df[existing_columns].fillna('').astype(str)

                # 각 행(row)별로 존재하는 컬럼 내용을 공백으로 이어붙임
                combined_series = df[existing_columns].apply(lambda row: ' '.join(row.values), axis=1)

                # 법령 전체의 텍스트를 하나의 문자열로 합침 (각 행은 줄바꿈으로 구분)
                full_text = '\n'.join(combined_series)

                # 출력 파일 경로 설정 (예: '가사소송법.csv' -> '가사소송법.txt')
                output_filename = filename.replace('.csv', '.txt')
                output_path = os.path.join(output_dir, output_filename)

                # 통합된 텍스트를 .txt 파일로 저장
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(full_text)

            except Exception as e:
                print(f"오류 발생: {filename} 처리 중 문제 발생 - {e}")

    print(f"\n작업 완료! 모든 법령이 '{output_dir}' 폴더에 .txt 파일로 저장되었습니다.")

'법령별 csv 저장 중복제거' 폴더의 CSV 파일들을 읽어 '법령_통합텍스트(현역)' 폴더에 TXT 파일로 통합합니다.
통합 대상 컬럼: ['조문내용', '항', '호', '목']
오류: 원본 폴더 '법령별 csv 저장 중복제거'를 찾을 수 없습니다. 경로를 확인해주세요.


In [2]:
# ==============================================================================
# 1. 라이브러리 및 환경 설정
# ==============================================================================
import os
import pandas as pd
import numpy as np
from tqdm import tqdm  # 진행 상황을 시각적으로 보여주는 라이브러리

# --- 텍스트 처리를 위한 라이브러리 ---
from konlpy.tag import Okt # Okt이 설치되지 않았다면 Okt로 변경: from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

# --- 딥러닝 및 유사도 계산을 위한 라이브러리 ---
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch

# tqdm을 pandas의 apply와 함께 사용하기 위해 필요한 설정
tqdm.pandas()

# Okt 초기화 (Okt 설치 경로에 따라 필요시 인자 추가)
# okt를 사용할 경우: tokenizer = Okt()
try:
    tokenizer = Okt()
    print("Okt 토크나이저를 사용합니다.")
except Exception as e:
    print(f"Okt 로드 실패: {e}")
    print("Okt 토크나이저로 대체합니다. 처리 속도가 느릴 수 있습니다.")
    from konlpy.tag import Okt
    tokenizer = Okt()


c:\Users\Javier\anaconda3\envs\legal_text\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Okt 토크나이저를 사용합니다.


In [3]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
# (이하 필요한 라이브러리들)

try:
    # ★ 탭으로 분리된 과목 목록 파일을 읽도록 sep='\t' 추가
    public_laws = pd.read_csv('변호사시험_출제대상_부속법령(공법).csv')['법령명'].tolist()
    civil_laws = pd.read_csv('변호사시험_출제대상_부속법령(민사법).csv')['법령명'].tolist()
    criminal_laws = pd.read_csv('변호사시험_출제대상_부속법령(형사법).csv')['법령명'].tolist()
    print("과목별 법령 목록 로드 완료.")
except FileNotFoundError as e:
    print(f"오류: 과목별 법령 목록 파일을 찾을 수 없습니다. ({e.filename})")
    exit() # 스크립트에서는 주석 해제

# --- 전체 법령 데이터 로드 (생성된 .txt 파일 로드) ---
all_laws_dir = '법령_통합텍스트(현역)' # ★ .txt 파일들이 저장된 폴더
all_laws_data = []


print(f"'{all_laws_dir}' 폴더에서 전체 법령 텍스트를 로드합니다...")
if not os.path.isdir(all_laws_dir):
    print(f"오류: '{all_laws_dir}' 폴더를 찾을 수 없습니다. 1단계 코드를 먼저 실행했는지 확인해주세요.")
    exit()

for file_name in tqdm(os.listdir(all_laws_dir)):
    if file_name.endswith('.txt'):
        law_name = file_name[:-4]  # '.txt' 확장자 제거
        file_path = os.path.join(all_laws_dir, file_name)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            all_laws_data.append({'법령명': law_name, '내용': content})
        except Exception as e:
            print(f"에러: {file_name} 파일 처리 중 오류 발생 - {e}")

df_all_laws = pd.DataFrame(all_laws_data)
print(f"전체 법령 데이터 로드 완료: 총 {len(df_all_laws)}개 법령")


과목별 법령 목록 로드 완료.
'법령_통합텍스트(현역)' 폴더에서 전체 법령 텍스트를 로드합니다...


100%|██████████| 5502/5502 [01:27<00:00, 62.60it/s]

전체 법령 데이터 로드 완료: 총 5502개 법령


In [4]:
# ==============================================================================
# 3. 데이터 전처리 (형태소 분석 및 명사 추출)
# ==============================================================================
print("\n--- 3. 데이터 전처리 시작 (명사 추출) ---")

def preprocess_text(text):
    """Mecab을 이용해 텍스트에서 명사만 추출하고 한 글자 단어는 제거"""
    try:
        # Mecab 사용 시: nouns = tokenizer.nouns(text)
        # Okt 사용 시: nouns = tokenizer.nouns(text)
        nouns = tokenizer.nouns(str(text))
        # 한 글자 명사 제거 후 공백으로 연결
        return ' '.join([noun for noun in nouns if len(noun) > 1])
    except Exception as e:
        # print(f"전처리 오류 발생: {e}, 원문: {text[:30]}")
        return "" # 오류 발생 시 빈 문자열 반환

# '내용' 컬럼에 전처리 함수 적용. 시간이 다소 소요될 수 있습니다.

tqdm.pandas()

df_all_laws['내용_전처리'] = df_all_laws['내용'].progress_apply(preprocess_text)
print("데이터 전처리 완료.")
print(df_all_laws[['법령명', '내용_전처리']].head())



--- 3. 데이터 전처리 시작 (명사 추출) ---


 81%|████████  | 4458/5502 [30:32<07:09,  2.43it/s]  


KeyboardInterrupt: 

In [ ]:
# 2. 비어있지 않은 법령 텍스트 하나를 샘플로 선택
sample_text = ""
for text in df_all_laws['내용']:
    if text and text.strip():
        sample_text = text
        break

if not sample_text:
    print("오류: 데이터프레임의 '내용' 컬럼이 모두 비어있습니다. 파일 로딩 과정을 다시 확인해주세요.")
else:
    print("="*50)
    print("              🔍 전처리 과정 진단 시작")
    print("="*50)

    # --- 진단 시작 ---
    print("\n[1단계] 원본 텍스트 샘플 (앞 200자):")
    print(sample_text[:200])

    # 형태소 분석기 초기화
    try:
        tokenizer = Mecab()
        print("\n[2단계] Mecab 형태소 분석기를 사용합니다.")
    except:
        from konlpy.tag import Okt
        tokenizer = Okt()
        print("\n[2단계] Okt 형태소 분석기를 사용합니다.")

    # 명사 추출 실행
    nouns_list = tokenizer.nouns(sample_text)
    print("\n[3단계] 형태소 분석기가 추출한 '명사' 목록 (앞 30개):")
    print(nouns_list[:30])

    # 한 글자 명사 제거
    filtered_nouns = [noun for noun in nouns_list if len(noun) > 1]
    print("\n[4단계] 한 글자를 제외하고 남은 '명사' 목록 (앞 30개):")
    print(filtered_nouns[:30])

    # 최종 결과
    final_result = " ".join(filtered_nouns)
    print("\n[5단계] 최종 전처리 결과 (앞 200자):")
    print(final_result[:200])
    
    if not final_result:
        print("\n🚨 진단 결과: 최종 결과가 비어있습니다.")
        if not nouns_list:
            print("-> 원인: 3단계에서 형태소 분석기가 명사를 전혀 추출하지 못했습니다. KoNLPy나 Mecab 설치가 올바른지 확인해주세요.")
        elif not filtered_nouns:
            print("-> 원인: 4단계에서 모든 명사가 한 글자라 제거되었습니다. 법령 텍스트 특성일 수 있으나, 3단계 결과가 이상하다면 토크나이저 문제를 의심해봐야 합니다.")

              🔍 전처리 과정 진단 시작

[1단계] 원본 텍스트 샘플 (앞 200자):
제1조(목적) 이 규칙은 「공증인법」 제13조의2, 제15조의6, 제24조, 제75조 및 제77조의9제1항에 따른 공증사무소의 시설 기준, 서류의 보존 방법, 보존 장소, 보존 기간, 폐기, 서류인계의 기준, 서류 통합보관시설의 기준 및 허가 절차, 그 밖의 설비에 관하여 필요한 사항을 규정함을 목적으로 한다.
   
제2조(정의) 이 규칙에서 서류란 「공

[2단계] Okt 형태소 분석기를 사용합니다.

[3단계] 형태소 분석기가 추출한 '명사' 목록 (앞 30개):
['제', '목적', '이', '규칙', '공증인', '법', '제', '제', '제', '제', '및', '제', '제', '항', '공증', '사무소', '시설', '기준', '서류', '보존', '방법', '보존', '장소', '보존', '기간', '폐기', '서류', '인계', '기준', '서류']

[4단계] 한 글자를 제외하고 남은 '명사' 목록 (앞 30개):
['목적', '규칙', '공증인', '공증', '사무소', '시설', '기준', '서류', '보존', '방법', '보존', '장소', '보존', '기간', '폐기', '서류', '인계', '기준', '서류', '통합', '보관', '시설', '기준', '허가', '절차', '설비', '사항', '규정', '목적', '정의']

[5단계] 최종 전처리 결과 (앞 200자):
목적 규칙 공증인 공증 사무소 시설 기준 서류 보존 방법 보존 장소 보존 기간 폐기 서류 인계 기준 서류 통합 보관 시설 기준 허가 절차 설비 사항 규정 목적 정의 규칙 서류 공증인 이하 서류 마이크로필름 전산 정보처리 조직 이하 전산 정보처리 조직 보존 서류 서류 보존 방법 증서 원본 사서 증서 인증서 사본 정관 법인 의사록 표지 붙이 증서 번호 등부 번


In [ ]:
# # --- ★★★ 전처리 결과 저장 코드 (이 부분을 추가하세요) ★★★ ---
# # 필요한 컬럼만 선택
# df_to_save = df_all_laws[['법령명', '내용', '내용_전처리']]

# # CSV 파일로 저장 (index=False는 불필요한 인덱스 컬럼이 생기는 것을 방지)
# output_filename = 'preprocessed_laws.csv'
# df_to_save.to_csv(output_filename, index=False, encoding='utf-8-sig')

# print(f"전처리 결과가 '{output_filename}' 파일로 저장되었습니다.")

전처리 결과가 'preprocessed_laws.csv' 파일로 저장되었습니다.


In [ ]:
# import os
# from tqdm import tqdm

# # 전처리된 파일들을 저장할 새 폴더 이름
# preprocessed_output_dir = '법령_전처리_텍스트' 

# # 새 폴더 생성 (이미 있으면 넘어감)
# os.makedirs(preprocessed_output_dir, exist_ok=True)
# print(f"\n전처리된 텍스트를 '{preprocessed_output_dir}' 폴더에 개별 저장합니다...")

# # 데이터프레임을 한 줄씩 순회하며 파일로 저장
# # df_all_laws는 전처리가 완료된 데이터프레임입니다.
# for index, row in tqdm(df_all_laws.iterrows(), total=df_all_laws.shape[0]):
#     law_name = row['법령명']
#     preprocessed_content = row['내용_전처리']
    
#     # 출력 파일 경로 설정
#     output_path = os.path.join(preprocessed_output_dir, f"{law_name}.txt")
    
#     # 전처리된 내용을 .txt 파일로 저장
#     with open(output_path, 'w', encoding='utf-8') as f:
#         f.write(preprocessed_content)

# print(f"\n작업 완료! 모든 전처리된 법령이 '{preprocessed_output_dir}' 폴더에 저장되었습니다.")


전처리된 텍스트를 '법령_전처리_텍스트' 폴더에 개별 저장합니다...


100%|██████████| 5502/5502 [00:11<00:00, 466.25it/s]


작업 완료! 모든 전처리된 법령이 '법령_전처리_텍스트' 폴더에 저장되었습니다.


In [ ]:
# --- 3. 데이터 전처리 ---
print("\n--- 데이터 전처리 시작 ---")
df_all_laws['내용_전처리'] = df_all_laws['내용'].progress_apply(preprocess_text)
print("-> 데이터 전처리 완료.")

# --- 4. 전처리 결과를 개별 txt 파일로 저장 ---
preprocessed_output_dir = '법령_전처리_텍스트'
os.makedirs(preprocessed_output_dir, exist_ok=True)
print(f"\n-> 전처리된 텍스트를 '{preprocessed_output_dir}' 폴더에 개별 저장합니다...")

try:
    for index, row in tqdm(df_all_laws.iterrows(), total=df_all_laws.shape[0]):
        law_name = row['법령명']
        preprocessed_content = row['내용_전처리']
        output_path = os.path.join(preprocessed_output_dir, f"{law_name}.txt")
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(preprocessed_content)
    print(f"\n✅ 저장 성공! '{preprocessed_output_dir}' 폴더를 확인해보세요.")
except Exception as e:
    print(f"🚨 저장 실패: {e}")


--- 데이터 전처리 시작 ---


100%|██████████| 5502/5502 [31:23<00:00,  2.92it/s]  


-> 데이터 전처리 완료.

-> 전처리된 텍스트를 '법령_전처리_텍스트' 폴더에 개별 저장합니다...


100%|██████████| 5502/5502 [00:15<00:00, 363.87it/s]


✅ 저장 성공! '법령_전처리_텍스트' 폴더를 확인해보세요.


### 유사도 분석함수

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def analyze_cluster_similarity(all_vectors, all_law_names, subject_law_list, subject_name):
    """
    주어진 과목 클러스터의 내부 유사도와 외부 유사도를 계산하는 함수.
    """
    print(f"\n[{subject_name}] 클러스터 유사도 분석 중...")
    subject_indices = all_law_names[all_law_names.isin(subject_law_list)].index.tolist()
    external_indices = all_law_names[~all_law_names.isin(subject_law_list)].index.tolist()

    if len(subject_indices) == 0:
        return 0, 0

    subject_vectors = all_vectors[subject_indices]
    external_vectors = all_vectors[external_indices]
    
    internal_sim_avg = 0
    if subject_vectors.shape[0] > 1:
      internal_sim_matrix = cosine_similarity(subject_vectors)
      internal_sim_avg = np.mean(internal_sim_matrix[np.triu_indices_from(internal_sim_matrix, k=1)])

    external_sim_avg = 0
    if len(external_indices) > 0 and len(subject_indices) > 0:
      external_sim_matrix = cosine_similarity(subject_vectors, external_vectors)
      external_sim_avg = np.mean(external_sim_matrix)

    print(f" - 내부 유사도: {internal_sim_avg:.4f}, 외부 유사도: {external_sim_avg:.4f}")
    return internal_sim_avg, external_sim_avg

# TF-IDF 분석

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# --- 4-1. TF-IDF 벡터화 ---
print("\n--- TF-IDF 분석 시작 ---")
tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=5)
tfidf_vectors = tfidf_vectorizer.fit_transform(df_all_laws['내용_전처리'])
print("TF-IDF 벡터화 완료. 벡터 shape:", tfidf_vectors.shape)

# --- 6-1. TF-IDF 유사도 분석 실행 ---
tfidf_results = []
subjects = {"공법": public_laws, "민사법": civil_laws, "형사법": criminal_laws}
all_law_names_series = df_all_laws['법령명']

for subject_name, law_list in subjects.items():
    internal_sim, external_sim = analyze_cluster_similarity(
        tfidf_vectors, all_law_names_series, law_list, f"{subject_name} (TF-IDF)"
    )
    tfidf_results.append({
        "과목명": subject_name,
        "내부 유사도": internal_sim,
        "외부 유사도": external_sim
    })

# --- 7-1. TF-IDF 최종 결과 출력 ---
print("\n\n" + "="*30)
print("     TF-IDF 분석 결과")
print("="*30)
df_tfidf_results = pd.DataFrame(tfidf_results).set_index('과목명')
print(df_tfidf_results)


--- TF-IDF 분석 시작 ---
TF-IDF 벡터화 완료. 벡터 shape: (5502, 5000)

[공법 (TF-IDF)] 클러스터 유사도 분석 중...
 - 내부 유사도: 0.0946, 외부 유사도: 0.0538

[민사법 (TF-IDF)] 클러스터 유사도 분석 중...
 - 내부 유사도: 0.1214, 외부 유사도: 0.0445

[형사법 (TF-IDF)] 클러스터 유사도 분석 중...
 - 내부 유사도: 0.1103, 외부 유사도: 0.0520


     TF-IDF 분석 결과
       내부 유사도    외부 유사도
과목명                    
공법   0.094576  0.053769
민사법  0.121437  0.044462
형사법  0.110265  0.052035


In [ ]:
df_tfidf_results.to_csv("sbert_과목별_결과_현역.idf", encoding = 'utf-8')

### SBERT 분석

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# --- 1. 원본 텍스트 데이터 로드 ---
# SBERT는 원본 텍스트를 사용하므로, 원본 .txt 파일이 있는 폴더를 지정합니다.
original_text_dir = '법령_통합텍스트(현역)' 
all_laws_data = []

print(f"'{original_text_dir}' 폴더에서 원본 법령 텍스트를 로드합니다...")
if not os.path.isdir(original_text_dir):
    print(f"오류: '{original_text_dir}' 폴더를 찾을 수 없습니다. 원본 txt 통합 파일이 준비되어 있는지 확인해주세요.")
    exit()

for filename in tqdm(os.listdir(original_text_dir)):
    if filename.endswith('.txt'):
        law_name = filename[:-4]
        file_path = os.path.join(original_text_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        all_laws_data.append({
            '법령명': law_name,
            '내용': content  # SBERT는 전처리 전 '내용' 컬럼을 사용
        })

df_all_laws = pd.DataFrame(all_laws_data)
print(f"\n원본 데이터 로딩 완료: 총 {len(df_all_laws)}개 법령")


# --- 2. 과목별 법령 목록 로드 ---
try:
    public_laws = pd.read_csv('변호사시험_출제대상_부속법령(공법).csv', sep='\t')['법령명'].tolist()
    civil_laws = pd.read_csv('변호사시험_출제대상_부속법령(민사법).csv', sep='\t')['법령명'].tolist()
    criminal_laws = pd.read_csv('변호사시험_출제대상_부속법령(형사법).csv', sep='\t')['법령명'].tolist()
    print("과목별 법령 목록 로드 완료.")
except FileNotFoundError as e:
    print(f"오류: 과목별 법령 목록 파일을 찾을 수 없습니다. ({e.filename})")
    exit()


# --- 3. 유사도 분석 함수 정의 (TF-IDF 분석 때와 동일) ---
def analyze_cluster_similarity(all_vectors, all_law_names, subject_law_list, subject_name):
    print(f"\n[{subject_name}] 클러스터 유사도 분석 중...")
    subject_indices = all_law_names[all_law_names.isin(subject_law_list)].index.tolist()
    external_indices = all_law_names[~all_law_names.isin(subject_law_list)].index.tolist()

    if len(subject_indices) == 0:
        return 0, 0

    subject_vectors = all_vectors[subject_indices]
    external_vectors = all_vectors[external_indices]

    internal_sim_avg = 0
    if subject_vectors.shape[0] > 1:
        internal_sim_matrix = cosine_similarity(subject_vectors)
        internal_sim_avg = np.mean(internal_sim_matrix[np.triu_indices_from(internal_sim_matrix, k=1)])

    external_sim_avg = 0
    if len(external_indices) > 0 and len(subject_indices) > 0:
        external_sim_matrix = cosine_similarity(subject_vectors, external_vectors)
        external_sim_avg = np.mean(external_sim_matrix)

    print(f" - 내부 유사도: {internal_sim_avg:.4f}, 외부 유사도: {external_sim_avg:.4f}")
    return internal_sim_avg, external_sim_avg


# --- 4. SBERT 분석 실행 ---
print("\n\n--- SBERT 분석 시작 ---")

# GPU 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"SBERT 분석에 사용할 장치: {device.upper()}")

# SBERT 모델 로드
print("SBERT 모델을 로드합니다. 최초 실행 시 수 분이 소요될 수 있습니다.")
sbert_model = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    device=device
)

print("SBERT 인코딩을 시작합니다. 법령 개수에 따라 많은 시간이 소요될 수 있습니다...")
sbert_vectors = sbert_model.encode(
    df_all_laws['내용'].tolist(), 
    show_progress_bar=True, 
    batch_size=32 
)
print("SBERT 벡터화 완료. 벡터 shape:", sbert_vectors.shape)


# --- 5. SBERT 유사도 계산 및 결과 취합 ---
sbert_results = []
subjects = {"공법": public_laws, "민사법": civil_laws, "형사법": criminal_laws}
all_law_names_series = df_all_laws['법령명']

for subject_name, law_list in subjects.items():
    internal_sim, external_sim = analyze_cluster_similarity(
        sbert_vectors, all_law_names_series, law_list, f"{subject_name} (SBERT)"
    )
    sbert_results.append({
        "과목명": subject_name,
        "내부 유사도": internal_sim,
        "외부 유사도": external_sim
    })

# --- 6. 최종 결과 출력 ---
print("\n\n" + "="*30)
print("      SBERT 분석 결과")
print("="*30)
df_sbert_results = pd.DataFrame(sbert_results).set_index('과목명')
print(df_sbert_results)

'법령_통합텍스트(현역)' 폴더에서 원본 법령 텍스트를 로드합니다...


100%|██████████| 5502/5502 [00:00<00:00, 5582.92it/s]



원본 데이터 로딩 완료: 총 5502개 법령
과목별 법령 목록 로드 완료.


--- SBERT 분석 시작 ---
SBERT 분석에 사용할 장치: CUDA
SBERT 모델을 로드합니다. 최초 실행 시 수 분이 소요될 수 있습니다.


c:\Users\silve\anaconda3\envs\nlp\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\silve\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

SBERT 인코딩을 시작합니다. 법령 개수에 따라 많은 시간이 소요될 수 있습니다...


Batches: 100%|██████████| 172/172 [00:23<00:00,  7.37it/s]

SBERT 벡터화 완료. 벡터 shape: (5502, 768)

[공법 (SBERT)] 클러스터 유사도 분석 중...
 - 내부 유사도: 0.7114, 외부 유사도: 0.6637

[민사법 (SBERT)] 클러스터 유사도 분석 중...
 - 내부 유사도: 0.7786, 외부 유사도: 0.6843

[형사법 (SBERT)] 클러스터 유사도 분석 중...
 - 내부 유사도: 0.6869, 외부 유사도: 0.6564


      SBERT 분석 결과
       내부 유사도    외부 유사도
과목명                    
공법   0.711350  0.663651
민사법  0.778606  0.684268
형사법  0.686883  0.656386


In [ ]:
df_sbert_results.to_csv("sbert_과목별_결과_현역.csv", encoding = 'utf-8')